In [14]:
import requests as re
import json
import pandas as pd

In [22]:
url = "http://localhost:8080"
endpoint_products = "/products/101"

In [23]:
response = re.get(url + endpoint_products)

In [24]:
returned_val = json.loads(response.content)

In [25]:
df = pd.DataFrame(returned_val, index=None)

In [26]:
df

,id,name,description,price,stock_quantity
0,101,Laptop Pro 1,Powerful laptop,55000,10


In [21]:
import unittest

# Test the returned json against the expected json(loaded from a csv file)
class TestJson(unittest.TestCase):
    def test_json(self):
        expected_json = pd.read_csv("products.csv")
        self.assertTrue(expected_json.equals(df))

unittest.main()


usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument '/home/sirjanh/.local/share/jupyter/runtime/kernel-v304b6c9ce2e1512b598b9fa7f920477ddaa37b066.json'


SystemExit: 2